In [1]:
import os
import math
import glob
import librosa
import time
import libf0
import statistics
import numpy as np
import pandas as pd
from scipy import signal
from skimage.restoration import denoise_tv_chambolle
from scipy.io import wavfile
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
from scipy.stats import ttest_1samp
from RecordingAnalysis4 import ExtractFeaturesRecordings

from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets

C:\Users\de Witasse Thézy\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\ProgramData\Anaconda3\lib\site-packages\libf0\salience.py:440: RuntimeWarning: invalid value encountered in cast
  bin_index = np.floor((1200 / R) * np.log2(F / F_ref) + 0.5).astype(np.int64)
C:\ProgramData\Anaconda3\lib\site-packages\libf0\salience.py:440: RuntimeWarning: invalid value encountered in cast
  bin_index = np.floor((1200 / R) * np.log2(F / F_ref) + 0.5).astype(np.int64)
C:\Users\de Witasse Thézy\AppData\Roaming\Python\Python38\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\de Witasse Thézy\AppData\Roaming\Python\Python38\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encou

In [24]:
basepath1 = "D:\\data\\playbackT01\\ch2"
basepath2 = "D:\\data\\playback2T01\\ch2"
basepath = basepath1
####

#reference_df = pd.read_csv("D:\\data\\playbackT01\\manual_duration.csv")
#reference_df.columns = ["Filename", "True_Duration"]
#reference_df['VocNumber'] = Freference_df.groupby('Filename').cumcount()

Freference_df = pd.read_csv("D:\\data\\playbackT01\\beg_end_freq.csv")
Freference_df.columns = ["Filename", "True_BeginningFrequency", "True_EndFrequency"]
Freference_df['VocNumber'] = Freference_df.groupby('Filename').cumcount()

In [22]:
def BetterOptimizationScore(params, X_train=None, y_train=None,reference_df=Freference_df,trade_off_factor=1):
    h, n, fmin,width = params['h'], params['n'], params['fmin'], params['width']
    weight, tol = 0.5, 9 #params.weight, params.width, params.tol

    file_paths = glob.glob(os.path.join(basepath, '*.wav'))

    start_time = time.time()
    generated_df = ExtractFeaturesRecordings(file_paths, h=h,n=n, fmin=fmin, weight=weight, width=width, tol=tol)
    execution_time = time.time() - start_time

    generated_df['Filename'] = generated_df['Filename'].astype(int)
    generated_df['VocNumber'] = generated_df.groupby('Filename').cumcount()
    # Merge with reference_df to fill missing lines with zero values
    merged_df = pd.merge(reference_df, generated_df, on=['Filename', 'VocNumber'], how='left').fillna(0)

    # Add 'actual' column
    merged_df['actual'] = merged_df['True_BeginningFrequency'] > 0
    # Add 'prediction' column
    merged_df['prediction'] = merged_df['BeginningFrequency'] > 0


    true_negatives = merged_df[(merged_df['actual'] == False) & (merged_df['prediction'] == False)]
    true_positives = merged_df[(merged_df['actual'] == True) & (merged_df['prediction'] == True)]

    #start_freq_mae = np.abs(true_positives['BeginningFrequency'] - true_positives['True_BeginningFrequency']).mean()
    #end_freq_mae = np.abs(true_positives['EndFrequency'] - true_positives['True_EndFrequency']).mean()

    start_freq_index = (np.abs(true_positives['BeginningFrequency'] - true_positives['True_BeginningFrequency'])/true_positives['True_BeginningFrequency']).mean()
    end_freq_index = (np.abs(true_positives['EndFrequency'] - true_positives['True_EndFrequency'])/true_positives['True_EndFrequency']).mean()


    # You can compute an overall accuracy index by averaging or combining the MAE values
    accuracy_index = (start_freq_index + end_freq_index) / 2.0
    combined_score = - accuracy_index - trade_off_factor * execution_time/len(file_paths)
    print(params, combined_score)

    return (combined_score, accuracy_index, execution_time/len(file_paths))

In [14]:
def classif_accuracy(params, X_train=None, y_train=None,reference_df=Freference_df,trade_off_factor=1):
    h, n, fmin = params['h'], params['n'], params['fmin']
    weight, width, tol = 0.5, 20, 9 #params.weight, params.width, params.tol
    
    file_paths = glob.glob(os.path.join(basepath, '*.wav'))
    
    start_time = time.time()
    generated_df = ExtractFeaturesRecordings(file_paths, h=h,n=n, fmin=fmin, weight=weight, width=width, tol=tol)
    execution_time = time.time() - start_time
    display(generated_df)
    generated_df['Filename'] = generated_df['Filename'].astype(int)
    generated_df['VocNumber'] = generated_df.groupby('Filename').cumcount()
    # Merge with reference_df to fill missing lines with zero values
    merged_df = pd.merge(reference_df, generated_df, on=['Filename', 'VocNumber'], how='left').fillna(0)
    
    # Add 'actual' column
    merged_df['actual'] = merged_df['True_BeginningFrequency'] > 0
    # Add 'prediction' column
    merged_df['prediction'] = merged_df['BeginningFrequency'] > 0

    true_negatives = merged_df[(merged_df['actual'] == False) & (merged_df['prediction'] == False)]
    true_positives = merged_df[(merged_df['actual'] == True) & (merged_df['prediction'] == True)]
    
    confusion_accuracy = (len(true_positives)+len(true_negatives))/len(merged_df)
    
    return confusion_accuracy

In [25]:
h=1100
n=1100
fmin=38141
width=5

BetterOptimizationScore({'h' : h, 'n' : n, 'fmin' : fmin,'width':width}, 
                        X_train=None, y_train=None,
                        reference_df=Freference_df,
                        trade_off_factor=1)

C:\ProgramData\Anaconda3\lib\site-packages\libf0\salience.py:440: RuntimeWarning: invalid value encountered in cast
  bin_index = np.floor((1200 / R) * np.log2(F / F_ref) + 0.5).astype(np.int64)
C:\Users\de Witasse Thézy\AppData\Roaming\Python\Python38\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\de Witasse Thézy\AppData\Roaming\Python\Python38\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{'h': 1100, 'n': 1100, 'fmin': 38141, 'width': 5} -0.07911127745927846


(-0.07911127745927846, 0.04510486728437882, 0.03400641017489963)

In [10]:
classif_accuracy({'h' : h, 'n' : n, 'fmin' : fmin}, 
                X_train=None, y_train=None,
                reference_df=Freference_df,
                trade_off_factor=1)

C:\ProgramData\Anaconda3\lib\site-packages\libf0\salience.py:440: RuntimeWarning: invalid value encountered in cast
  bin_index = np.floor((1200 / R) * np.log2(F / F_ref) + 0.5).astype(np.int64)


,Filename,Frequency,Duration,BeginningFrequency,EndFrequency
0,0000337,63110.776213,0.01736,64146.643288,62681.53193
1,0000341,65280.513400,0.02480,67569.574627,60197.64742


0.058823529411764705

In [7]:
file_paths = glob.glob(os.path.join(basepath, '*.wav'))
4.671241283416748/len(file_paths)

0.12975670231713188